# <a name="0">Image Classification With Neural Network</a>

1. <a href="#1">Data Processing</a>
2. <a href="#2">Build DataSet</a>
3. <a herf="#3">Build CNN</a>
4. <a herf="#4">Loss&Optimizer</a>
5. <a herf="#5">Training&Testing</a>

### 1. <a name="1">Data Processing</a>

- Locate the path to the image-dataset folder
- Transformation & Transportation
- Labeling

In [ ]:
#Importing the libraries
import glob
import torch
from torch.utils import data
from PIL import Image
import numpy as np
from torchvision import transforms
import torchvision
import matplotlib.pyplot as plt

In [ ]:
#Importing the dataset
all_imgs_path = glob.glob(r'./image_data/*/*.png')
print(all_imgs_path[:100])
print("Numbers of All Samples: ",len(all_imgs_path))

In [ ]:
#Set Labels
species = ['bagel','donut']
species_to_id = dict((c, i) for i, c in enumerate(species))
id_to_species = dict((v, k) for k, v in species_to_id.items())
print(id_to_species)
all_labels = []
#iter through the path of the image
for img in all_imgs_path:
    # Classify each img should be on which label
    for i, c in enumerate(species):
        if c in img:
            all_labels.append(i)
print(all_labels[:100]) #Get All labels

#### 1. <a name="1">Transpose the data</a>

Transform all photos to the same size: "96 * 96"
and then making sure they are all between the tensor[-1,1]

transform = transforms.Compose([
                transforms.Resize((96,96)), 
                transforms.ToTensor(), 
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])